#### In this document, we show how to use the FISH (FAST InterSterllar HI) scripts to calibrate the FAST CRAFTS data.

### Import necessary packages

In [1]:
import os
import numpy as np
import sys
import gc
import multiprocessing as mp
sys.path.append('/bb8/lingrui/my_projects/FAST_pipeline/FISH_scripts/')

In [2]:
!pwd

/bb8/lingrui/my_projects/FAST_pipeline


### Create soft links to the psr data and and spec data

In [3]:
os.system('rm org_psr org_spec')
os.system('ln -s /TheForce/fast_data/PT2021_0055_1/ org_psr')
os.system('ln -s /TheForce/fast_data/PT2021_0055_2/ org_spec')

0

### Prepare the output directories

In [2]:
Scan_name = ['Sun17_A_00','Sun17_A_01','Sun17_A_02','Sun17_A_03','Sun17_A_04','Sun17_A_05','Sun17_A_06','Sun17_A_07','Sun17_A_08','Sun17_A_09','Sun17_A_10']

In [3]:
Beam_name = ['M01','M02','M03','M04','M05','M06','M07','M08','M09','M10','M11','M12','M13','M14','M15','M16','M17','M18','M19']

In [ ]:
# ## 
# for i in Scan_name:
#     os.system('rm -rf {}'.format(i))      
#     os.system('mkdir {}'.format(i))
#     for j in Beam_name:
#         os.system('mkdir {}/{}'.format(i,j))
#         os.system('mkdir {}/{}/HighCAL'.format(i,j))
#         os.system('mkdir {}/{}/Calibrated'.format(i,j))

#### Generate PSR file list

In [ ]:
rootwd = os.getcwd()

for i in Scan_name:
    for j in Beam_name:
        os.system('ls $PWD/org_psr/{0}/*/*{1}*.fits > {0}/{1}/mypsrlist_{0}_{1}'.format(i,j))

#### Extract cal (Noise_On and Noise_Off)

In [ ]:
from extract_crafts_cal import extract_crafts_cal

rootwd = os.getcwd()

def target_extract_crafts_cal(i,j):
    os.chdir(rootwd+'/'+'{0}/{1}'.format(i,j))
    #print('Dealing with mypsrlist_{0}_{1}'.format(i,j))
    psrlist = np.loadtxt('mypsrlist_{0}_{1}'.format(i,j),dtype=str)
    for psrfits in psrlist:
        extract_crafts_cal(psrfits)

def run_process(i):
    process = []
    for j in Beam_name:
        process.append(mp.Process(target=target_extract_crafts_cal,args=(i,j)))
    [p.start() for p in process]
    [p.join()  for p in process]

for i in Scan_name:
    run_process(i)

#### Generate the cal_list

In [ ]:
for i in Scan_name:
    for j in Beam_name:
        os.system('ls $PWD/{0}/{1}/HighCAL/* > {0}/{1}/mycallist_{0}_{1}'.format(i,j))

#### Generate Tsys_psr


In [ ]:
from Tsys_cal import gaincal
!pwd

In [ ]:
Tcalfits = '/bb8/lingrui/my_projects/FAST_pipeline/FISH_scripts/Tcal/20201014/CAL.20201014.low.W.fits'
rootwd = os.getcwd()

def target_gaincal(i,j):
    os.chdir(rootwd+'/'+'{0}/{1}'.format(i,j))
    beam = int(j.split('M')[-1])
    print('Dealing with mycallist_{0}_{1}'.format(i,j))
    gaincal(callist='mycallist_{0}_{1}'.format(i,j),outname='Tsys_psr_{0}_{1}'.format(i,j),Tcalfits=Tcalfits,beam=beam,plot=True)
    print('Tsys_psr_{0}_{1} Created!'.format(i,j))
    gc.collect()

def run_process(j):
    process = []
    for i in Scan_name:
        process.append(mp.Process(target=target_gaincal,args=(i,j)))
    [p.start() for p in process]
    [p.join()  for p in process]

for j in Beam_name:
    run_process(j)

os.chdir(rootwd)

#### Generate speclist

In [ ]:
for i in Scan_name:
    for j in Beam_name:
        os.system('ls $PWD/org_spec/{0}/*/*{1}*N* > {0}/{1}/myspeclist_N_{0}_{1}'.format(i,j))

#### Apply cal

In [ ]:
from apply_cal import applycal
import numpy
import glob

rootwd = os.getcwd()

def target_applycal(i,j):
    os.chdir(rootwd+'/'+'{}/{}'.format(i,j))
    print('Dealing with myspeclist_N_{}_{}'.format(i,j))
    tsysfits = glob.glob('Tsys_psr*.fits')[0]
    speclist = np.loadtxt('myspeclist_N_{}_{}'.format(i,j),dtype=str)
    for specfits in speclist:
        applycal(specfits,tsysfits,restfreq=1.420405751768,plot=True)
        print('Calibrated file of {} Created!'.format(specfits))
    gc.collect()

def run_process(i):
    process = []
    for j in Beam_name:
        process.append(mp.Process(target=target_applycal,args=(i,j)))
    [p.start() for p in process]
    [p.join()  for p in process]

for i in Scan_name:
    run_process(i)

#### xyz2radec

In [ ]:
for i in Scan_name:
    for j in Beam_name:
        os.system('ls $PWD/{0}/{1}/Calibrated/*calibrated.fits > {0}/{1}/myspeclist_N_{0}_{1}_calibrated'.format(i,j))

from xyz2celestial import xyz2celestial
import glob
import pandas as pd

os.system('ln -s /TheForce/fast_data/PT2021_0055_KY PT2021_0055_KY')

In [ ]:
rootwd = os.getcwd()

def target_xyz2celestial(i,j,xyztable):
    os.chdir(rootwd+'/'+'{}/{}'.format(i,j))
    print('Dealing with myspeclist_N_{0}_{1}_calibrated'.format(i,j))
    beam = int(j.split('M')[-1])
    speclist = np.loadtxt('myspeclist_N_{0}_{1}_calibrated'.format(i,j),dtype=str)
    for specfits in speclist:
        xyz2celestial(specfits,xyztable,beam=beam,plot=False)

def run_process(i,xyztable):
    process = []
    for j in Beam_name:
        process.append(mp.Process(target=target_xyz2celestial,args=(i,j,xyztable)))
    [p.start() for p in process]
    [p.join()  for p in process]

for i in Scan_name:
    table = glob.glob(rootwd+'/*KY/'+'{}*.xlsx'.format(i))[0]
    xyztable = pd.read_excel(table,sheet_name=0)
    run_process(i,xyztable)

#### Ta to Tmb

In [5]:
for i in Scan_name:
    for j in Beam_name:
        os.system('ls $PWD/{0}/{1}/Calibrated/*calibrated_icrs.fits > {0}/{1}/myspeclist_N_{0}_{1}_calibrated_icrs'.format(i,j))

In [5]:
from Ta2Tmb import Ta2Tmb
import glob
import pandas as pd

In [6]:
help(Ta2Tmb)

Help on function Ta2Tmb in module Ta2Tmb:

Ta2Tmb(specfits, beam=1, restfreq=1420.405751768)



In [ ]:
rootwd = os.getcwd()

def target_Ta2Tmb(i,j):
    os.chdir(rootwd+'/'+'{}/{}'.format(i,j))
    print('Dealing with myspeclist_N_{0}_{1}_calibrated_icrs'.format(i,j))
    beam = int(j.split('M')[-1])
    speclist = np.loadtxt('myspeclist_N_{0}_{1}_calibrated_icrs'.format(i,j),dtype=str)
    for specfits in speclist:
        Ta2Tmb(specfits,beam=beam,restfreq=1420.405751768)

def run_process(i):
    process = []
    for j in Beam_name:
        process.append(mp.Process(target=target_Ta2Tmb,args=(i,j)))
    [p.start() for p in process]
    [p.join()  for p in process]

for i in Scan_name:
    run_process(i)

Dealing with myspeclist_N_Sun17_A_00_M01_calibrated_icrs
Dealing with myspeclist_N_Sun17_A_00_M02_calibrated_icrs
Dealing with myspeclist_N_Sun17_A_00_M03_calibrated_icrs
Dealing with myspeclist_N_Sun17_A_00_M04_calibrated_icrs
Dealing with myspeclist_N_Sun17_A_00_M05_calibrated_icrs
Dealing with myspeclist_N_Sun17_A_00_M06_calibrated_icrs
Dealing with myspeclist_N_Sun17_A_00_M07_calibrated_icrs
Dealing with myspeclist_N_Sun17_A_00_M08_calibrated_icrs
Dealing with myspeclist_N_Sun17_A_00_M09_calibrated_icrs
Dealing with myspeclist_N_Sun17_A_00_M10_calibrated_icrs
Dealing with myspeclist_N_Sun17_A_00_M11_calibrated_icrs
Dealing with myspeclist_N_Sun17_A_00_M12_calibrated_icrs
Dealing with myspeclist_N_Sun17_A_00_M13_calibrated_icrs
Dealing with myspeclist_N_Sun17_A_00_M14_calibrated_icrsDealing with myspeclist_N_Sun17_A_00_M15_calibrated_icrs


Dealing with myspeclist_N_Sun17_A_00_M16_calibrated_icrsDealing with myspeclist_N_Sun17_A_00_M17_calibrated_icrsDealing with myspeclist_N_Sun17_

/home/lingrui/.local/lib/python3.10/site-packages/astropy/time/utils.py:179: RuntimeWarning: overflow encountered in double_scalars
  c = 134217729. * a  # 2**27+1.
/home/lingrui/.local/lib/python3.10/site-packages/astropy/time/utils.py:181: RuntimeWarning: invalid value encountered in double_scalars
  ah = c - abig
/home/lingrui/.local/lib/python3.10/site-packages/astropy/time/utils.py:179: RuntimeWarning: overflow encountered in double_scalars
  c = 134217729. * a  # 2**27+1.
/home/lingrui/.local/lib/python3.10/site-packages/astropy/time/utils.py:179: RuntimeWarning: overflow encountered in double_scalars
  c = 134217729. * a  # 2**27+1.
/home/lingrui/.local/lib/python3.10/site-packages/astropy/time/utils.py:181: RuntimeWarning: invalid value encountered in double_scalars
  ah = c - abig
/home/lingrui/.local/lib/python3.10/site-packages/astropy/time/utils.py:181: RuntimeWarning: invalid value encountered in double_scalars
  ah = c - abig
/home/lingrui/.local/lib/python3.10/site-packa

Dealing with myspeclist_N_Sun17_A_01_M01_calibrated_icrsDealing with myspeclist_N_Sun17_A_01_M02_calibrated_icrs

Dealing with myspeclist_N_Sun17_A_01_M03_calibrated_icrs
Dealing with myspeclist_N_Sun17_A_01_M04_calibrated_icrs
Dealing with myspeclist_N_Sun17_A_01_M05_calibrated_icrs
Dealing with myspeclist_N_Sun17_A_01_M06_calibrated_icrs
Dealing with myspeclist_N_Sun17_A_01_M07_calibrated_icrs
Dealing with myspeclist_N_Sun17_A_01_M08_calibrated_icrs

Dealing with myspeclist_N_Sun17_A_01_M09_calibrated_icrsDealing with myspeclist_N_Sun17_A_01_M10_calibrated_icrs
Dealing with myspeclist_N_Sun17_A_01_M11_calibrated_icrs
Dealing with myspeclist_N_Sun17_A_01_M12_calibrated_icrs
Dealing with myspeclist_N_Sun17_A_01_M13_calibrated_icrs
Dealing with myspeclist_N_Sun17_A_01_M14_calibrated_icrs
Dealing with myspeclist_N_Sun17_A_01_M15_calibrated_icrs
Dealing with myspeclist_N_Sun17_A_01_M16_calibrated_icrs
Dealing with myspeclist_N_Sun17_A_01_M17_calibrated_icrsDealing with myspeclist_N_Sun17_

/home/lingrui/.local/lib/python3.10/site-packages/astropy/time/utils.py:179: RuntimeWarning: overflow encountered in double_scalars
  c = 134217729. * a  # 2**27+1.
/home/lingrui/.local/lib/python3.10/site-packages/astropy/time/utils.py:181: RuntimeWarning: invalid value encountered in double_scalars
  ah = c - abig
/home/lingrui/.local/lib/python3.10/site-packages/astropy/time/utils.py:179: RuntimeWarning: overflow encountered in double_scalars
  c = 134217729. * a  # 2**27+1.
/home/lingrui/.local/lib/python3.10/site-packages/astropy/time/utils.py:181: RuntimeWarning: invalid value encountered in double_scalars
  ah = c - abig
/home/lingrui/.local/lib/python3.10/site-packages/astropy/time/utils.py:179: RuntimeWarning: overflow encountered in double_scalars
  c = 134217729. * a  # 2**27+1.
/home/lingrui/.local/lib/python3.10/site-packages/astropy/time/utils.py:181: RuntimeWarning: invalid value encountered in double_scalars
  ah = c - abig
/home/lingrui/.local/lib/python3.10/site-packa

Dealing with myspeclist_N_Sun17_A_02_M01_calibrated_icrs
Dealing with myspeclist_N_Sun17_A_02_M02_calibrated_icrs
Dealing with myspeclist_N_Sun17_A_02_M03_calibrated_icrs
Dealing with myspeclist_N_Sun17_A_02_M04_calibrated_icrs
Dealing with myspeclist_N_Sun17_A_02_M05_calibrated_icrs
Dealing with myspeclist_N_Sun17_A_02_M06_calibrated_icrs
Dealing with myspeclist_N_Sun17_A_02_M07_calibrated_icrs
Dealing with myspeclist_N_Sun17_A_02_M08_calibrated_icrs
Dealing with myspeclist_N_Sun17_A_02_M09_calibrated_icrs
Dealing with myspeclist_N_Sun17_A_02_M10_calibrated_icrs
Dealing with myspeclist_N_Sun17_A_02_M11_calibrated_icrs
Dealing with myspeclist_N_Sun17_A_02_M12_calibrated_icrs
Dealing with myspeclist_N_Sun17_A_02_M13_calibrated_icrs
Dealing with myspeclist_N_Sun17_A_02_M14_calibrated_icrs
Dealing with myspeclist_N_Sun17_A_02_M15_calibrated_icrs
Dealing with myspeclist_N_Sun17_A_02_M16_calibrated_icrs
Dealing with myspeclist_N_Sun17_A_02_M17_calibrated_icrsDealing with myspeclist_N_Sun17_

...

Dealing with myspeclist_N_Sun17_A_03_M01_calibrated_icrs
Dealing with myspeclist_N_Sun17_A_03_M02_calibrated_icrs
Dealing with myspeclist_N_Sun17_A_03_M03_calibrated_icrs
Dealing with myspeclist_N_Sun17_A_03_M04_calibrated_icrs
Dealing with myspeclist_N_Sun17_A_03_M05_calibrated_icrs
Dealing with myspeclist_N_Sun17_A_03_M06_calibrated_icrs
Dealing with myspeclist_N_Sun17_A_03_M07_calibrated_icrs
Dealing with myspeclist_N_Sun17_A_03_M08_calibrated_icrs
Dealing with myspeclist_N_Sun17_A_03_M09_calibrated_icrs

Dealing with myspeclist_N_Sun17_A_03_M10_calibrated_icrsDealing with myspeclist_N_Sun17_A_03_M11_calibrated_icrs
Dealing with myspeclist_N_Sun17_A_03_M12_calibrated_icrs
Dealing with myspeclist_N_Sun17_A_03_M13_calibrated_icrs
Dealing with myspeclist_N_Sun17_A_03_M14_calibrated_icrs
Dealing with myspeclist_N_Sun17_A_03_M15_calibrated_icrs
Dealing with myspeclist_N_Sun17_A_03_M16_calibrated_icrs
Dealing with myspeclist_N_Sun17_A_03_M17_calibrated_icrs
Dealing with myspeclist_N_Sun17

/home/lingrui/.local/lib/python3.10/site-packages/astropy/time/utils.py:179: RuntimeWarning: overflow encountered in double_scalars
  c = 134217729. * a  # 2**27+1.


/home/lingrui/.local/lib/python3.10/site-packages/astropy/time/utils.py:181: RuntimeWarning: invalid value encountered in double_scalars
  ah = c - abig


Dealing with myspeclist_N_Sun17_A_03_M19_calibrated_icrs


/home/lingrui/.local/lib/python3.10/site-packages/astropy/time/utils.py:179: RuntimeWarning: overflow encountered in double_scalars
  c = 134217729. * a  # 2**27+1.
/home/lingrui/.local/lib/python3.10/site-packages/astropy/time/utils.py:181: RuntimeWarning: invalid value encountered in double_scalars
  ah = c - abig
/home/lingrui/.local/lib/python3.10/site-packages/astropy/time/utils.py:179: RuntimeWarning: overflow encountered in double_scalars
  c = 134217729. * a  # 2**27+1.
/home/lingrui/.local/lib/python3.10/site-packages/astropy/time/utils.py:181: RuntimeWarning: invalid value encountered in double_scalars
  ah = c - abig
/home/lingrui/.local/lib/python3.10/site-packages/astropy/time/utils.py:179: RuntimeWarning: overflow encountered in double_scalars
  c = 134217729. * a  # 2**27+1.
/home/lingrui/.local/lib/python3.10/site-packages/astropy/time/utils.py:181: RuntimeWarning: invalid value encountered in double_scalars
  ah = c - abig
/home/lingrui/.local/lib/python3.10/site-packa

#### Gridding 

In [8]:
# os.system('ls $PWD/Sun17_A_00/*/Calibrated/*_calibrated_icrs.fits \
#               $PWD/Sun17_A_01/*/Calibrated/*_calibrated_icrs.fits \
#               $PWD/Sun17_A_02/*/Calibrated/*_calibrated_icrs.fits \
#               $PWD/Sun17_A_03/*/Calibrated/*_calibrated_icrs.fits \
#               $PWD/Sun17_A_04/*/Calibrated/*_calibrated_icrs.fits > mygridlist')

os.system('ls $PWD/*/*/Calibrated/*_calibrated_icrs_Tmb.fits > mygridlist_all')

0

In [9]:
# from gridding import gridding

# speclist = 'mygridlist'
# center = [284.0,6.0]
# gridding(speclist=speclist, center=center, imsize=[300, 300], cell=1/60, rest_freq=1.420405751768, vmin=-200, vmax=100, p=0, outname='Sun17_A_scans.fits', overwrite=False)

In [10]:
from gridding_with_doppler_tracking import gridding_with_doppler_tracking

speclist = 'mygridlist_all'
center = [284.5,6.5]
outname = 'Sun17_A_scans_all_doppler_Tmb.fits'
gridding_with_doppler_tracking(speclist=speclist, center=center, imsize=[360, 360], cell=1/60, rest_freq=1.420405751768, vmin=-300, vmax=300, dv=0.5, p=0,  outname=outname, overwrite=False)

{'NAXIS': 2, 'NAXIS1': 360, 'NAXIS2': 360, 'CTYPE1': 'RA---SIN', 'CTYPE2': 'DEC--SIN', 'CUNIT1': 'deg', 'CUNIT2': 'deg', 'CDELT1': -0.016666666666666666, 'CDELT2': 0.016666666666666666, 'CRPIX1': 180.0, 'CRPIX2': 180.0, 'CRVAL1': 284.5, 'CRVAL2': 6.5}
Gridding /bb8/lingrui/my_projects/FAST_pipeline/Sun17_A_00/M01/Calibrated/Sun17_A_00_arcdrift-M01_N_0001_211010_100500to211010_101152_calibrated_icrs_Tmb.fits
Doing Doppler Tracking!


/home/lingrui/.local/lib/python3.10/site-packages/astropy/time/utils.py:179: RuntimeWarning: overflow encountered in double_scalars
  c = 134217729. * a  # 2**27+1.
/home/lingrui/.local/lib/python3.10/site-packages/astropy/time/utils.py:181: RuntimeWarning: invalid value encountered in double_scalars
  ah = c - abig


Doing Doppler Tracking!


Gridding /bb8/lingrui/my_projects/FAST_pipeline/Sun17_A_00/M15/Calibrated/Sun17_A_00_arcdrift-M15_N_0002_211010_101152to211010_101844_calibrated_icrs_Tmb.fits
Doing Doppler Tracking!


Gridding /bb8/lingrui/my_projects/FAST_pipeline/Sun17_A_00/M15/Calibrated/Sun17_A_00_arcdrift-M15_N_0003_211010_101844to211010_101848_calibrated_icrs_Tmb.fits
Doing Doppler Tracking!


...

Gridding /bb8/lingrui/my_projects/FAST_pipeline/Sun17_A_10/M18/Calibrated/Sun17_A_10_arcdrift-M18_N_0003_211119_072344to211119_072348_calibrated_icrs_Tmb.fits
Doing Doppler Tracking!


Gridding /bb8/lingrui/my_projects/FAST_pipeline/Sun17_A_10/M19/Calibrated/Sun17_A_10_arcdrift-M19_N_0001_211119_071000to211119_071652_calibrated_icrs_Tmb.fits
Doing Doppler Tracking!


Gridding /bb8/lingrui/my_projects/FAST_pipeline/Sun17_A_10/M19/Calibrated/Sun17_A_10_arcdrift-M19_N_0002_211119_071652to211119_072344_calibrated_icrs_Tmb.fits
Doing Doppler Tracking!


Gridding /bb8/lingrui/my_projects/FAST_pipeline/Sun17_A_10/M19/Calibrated/Sun17_A_10_arcdrift-M19_N_0003_211119_072344to211119_072348_calibrated_icrs_Tmb.fits
Doing Doppler Tracking!


/bb8/lingrui/my_projects/FAST_pipeline/FISH_scripts/gridding_with_doppler_tracking.py:129: RuntimeWarning: invalid value encountered in true_divide
  gridded_data = gridder.get_datacube()


# Wideband continuum

#### xyz2radec

In [5]:
from xyz2celestial import xyz2celestial
import glob
import pandas as pd

os.system('ln -s /TheForce/fast_data/PT2021_0055_KY PT2021_0055_KY')

ln: failed to create symbolic link 'PT2021_0055_KY/PT2021_0055_KY': File exists


256

In [6]:
rootwd = os.getcwd()

def target_xyz2celestial(i,j,xyztable):
    os.chdir(rootwd+'/'+'{}/{}'.format(i,j))
    beam = int(j.split('M')[-1])
    specfits = glob.glob('Tsys_psr*.fits')[0]
    print('Dealing with '+specfits)
    xyz2celestial(specfits,xyztable,beam=beam,plot=False,psrflag=True)

def run_process(i,xyztable):
    process = []
    for j in Beam_name:
        process.append(mp.Process(target=target_xyz2celestial,args=(i,j,xyztable)))
    [p.start() for p in process]
    [p.join()  for p in process]

for i in Scan_name:
    table = glob.glob(rootwd+'/*KY/'+'{}*.xlsx'.format(i))[0]
    xyztable = pd.read_excel(table,sheet_name=0)
    run_process(i,xyztable)

Dealing with Tsys_psr_Sun17_A_00_M01-M01_211010_100500to211010_101848.fits
Dealing with Tsys_psr_Sun17_A_00_M02-M02_211010_100500to211010_101848.fits
Dealing with Tsys_psr_Sun17_A_00_M03-M03_211010_100500to211010_101848.fits
Dealing with Tsys_psr_Sun17_A_00_M04-M04_211010_100500to211010_101848.fits
Dealing with Tsys_psr_Sun17_A_00_M05-M05_211010_100500to211010_101848.fits
Dealing with Tsys_psr_Sun17_A_00_M06-M06_211010_100500to211010_101848.fits
Dealing with Tsys_psr_Sun17_A_00_M07-M07_211010_100500to211010_101848.fits
Dealing with Tsys_psr_Sun17_A_00_M08-M08_211010_100500to211010_101848.fits
Dealing with Tsys_psr_Sun17_A_00_M09-M09_211010_100500to211010_101848.fits
Dealing with Tsys_psr_Sun17_A_00_M10-M10_211010_100500to211010_101848.fits
Dealing with Tsys_psr_Sun17_A_00_M11-M11_211010_100500to211010_101848.fits
Dealing with Tsys_psr_Sun17_A_00_M12-M12_211010_100500to211010_101848.fits
Dealing with Tsys_psr_Sun17_A_00_M13-M13_211010_100500to211010_101848.fits
Dealing with Tsys_psr_Sun

/home/lingrui/.local/lib/python3.10/site-packages/astropy/time/utils.py:179: RuntimeWarning: overflow encountered in double_scalars
  c = 134217729. * a  # 2**27+1.
/home/lingrui/.local/lib/python3.10/site-packages/astropy/time/utils.py:181: RuntimeWarning: invalid value encountered in double_scalars
  ah = c - abig


Dealing with Tsys_psr_Sun17_A_00_M18-M18_211010_100500to211010_101848.fits
Dealing with Tsys_psr_Sun17_A_00_M19-M19_211010_100500to211010_101848.fits


/home/lingrui/.local/lib/python3.10/site-packages/astropy/time/utils.py:179: RuntimeWarning: overflow encountered in double_scalars
  c = 134217729. * a  # 2**27+1.
/home/lingrui/.local/lib/python3.10/site-packages/astropy/time/utils.py:181: RuntimeWarning: invalid value encountered in double_scalars
  ah = c - abig
/home/lingrui/.local/lib/python3.10/site-packages/astropy/time/utils.py:179: RuntimeWarning: overflow encountered in double_scalars
  c = 134217729. * a  # 2**27+1.
/home/lingrui/.local/lib/python3.10/site-packages/astropy/time/utils.py:181: RuntimeWarning: invalid value encountered in double_scalars
  ah = c - abig
/home/lingrui/.local/lib/python3.10/site-packages/astropy/time/utils.py:179: RuntimeWarning: overflow encountered in double_scalars
  c = 134217729. * a  # 2**27+1.
/home/lingrui/.local/lib/python3.10/site-packages/astropy/time/utils.py:179: RuntimeWarning: overflow encountered in double_scalars
  c = 134217729. * a  # 2**27+1.
/home/lingrui/.local/lib/python3.1

Dealing with Tsys_psr_Sun17_A_01_M01-M01_211010_103100to211010_104448.fits
Dealing with Tsys_psr_Sun17_A_01_M02-M02_211010_103100to211010_104448.fits
Dealing with Tsys_psr_Sun17_A_01_M03-M03_211010_103100to211010_104448.fits
Dealing with Tsys_psr_Sun17_A_01_M04-M04_211010_103100to211010_104448.fits
Dealing with Tsys_psr_Sun17_A_01_M05-M05_211010_103100to211010_104448.fits
Dealing with Tsys_psr_Sun17_A_01_M06-M06_211010_103100to211010_104448.fits
Dealing with Tsys_psr_Sun17_A_01_M07-M07_211010_103100to211010_104448.fits
Dealing with Tsys_psr_Sun17_A_01_M08-M08_211010_103100to211010_104448.fits
Dealing with Tsys_psr_Sun17_A_01_M09-M09_211010_103100to211010_104448.fits
Dealing with Tsys_psr_Sun17_A_01_M10-M10_211010_103100to211010_104448.fits
Dealing with Tsys_psr_Sun17_A_01_M11-M11_211010_103100to211010_104448.fits
Dealing with Tsys_psr_Sun17_A_01_M12-M12_211010_103100to211010_104448.fits
Dealing with Tsys_psr_Sun17_A_01_M13-M13_211010_103100to211010_104448.fits
Dealing with Tsys_psr_Sun

...

/home/lingrui/.local/lib/python3.10/site-packages/astropy/time/utils.py:179: RuntimeWarning: overflow encountered in double_scalars
  c = 134217729. * a  # 2**27+1.
/home/lingrui/.local/lib/python3.10/site-packages/astropy/time/utils.py:181: RuntimeWarning: invalid value encountered in double_scalars
  ah = c - abig
/home/lingrui/.local/lib/python3.10/site-packages/astropy/time/utils.py:179: RuntimeWarning: overflow encountered in double_scalars
  c = 134217729. * a  # 2**27+1.
/home/lingrui/.local/lib/python3.10/site-packages/astropy/time/utils.py:181: RuntimeWarning: invalid value encountered in double_scalars
  ah = c - abig
/home/lingrui/.local/lib/python3.10/site-packages/astropy/time/utils.py:179: RuntimeWarning: overflow encountered in double_scalars
  c = 134217729. * a  # 2**27+1.
/home/lingrui/.local/lib/python3.10/site-packages/astropy/time/utils.py:181: RuntimeWarning: invalid value encountered in double_scalars
  ah = c - abig
/home/lingrui/.local/lib/python3.10/site-packa

Dealing with Tsys_psr_Sun17_A_09_M01-M01_211023_110300to211023_111648.fits
Dealing with Tsys_psr_Sun17_A_09_M02-M02_211023_110300to211023_111648.fits
Dealing with Tsys_psr_Sun17_A_09_M03-M03_211023_110300to211023_111648.fits
Dealing with Tsys_psr_Sun17_A_09_M04-M04_211023_110300to211023_111648.fits
Dealing with Tsys_psr_Sun17_A_09_M05-M05_211023_110300to211023_111648.fits
Dealing with Tsys_psr_Sun17_A_09_M06-M06_211023_110300to211023_111648.fits
Dealing with Tsys_psr_Sun17_A_09_M07-M07_211023_110300to211023_111648.fits
Dealing with Tsys_psr_Sun17_A_09_M08-M08_211023_110300to211023_111648.fits
Dealing with Tsys_psr_Sun17_A_09_M09-M09_211023_110300to211023_111648.fits
Dealing with Tsys_psr_Sun17_A_09_M10-M10_211023_110300to211023_111648.fitsDealing with Tsys_psr_Sun17_A_09_M11-M11_211023_110300to211023_111648.fits

Dealing with Tsys_psr_Sun17_A_09_M12-M12_211023_110300to211023_111648.fits
Dealing with Tsys_psr_Sun17_A_09_M13-M13_211023_110300to211023_111648.fits
Dealing with Tsys_psr_Sun

/home/lingrui/.local/lib/python3.10/site-packages/astropy/time/utils.py:179: RuntimeWarning: overflow encountered in double_scalars
  c = 134217729. * a  # 2**27+1.
/home/lingrui/.local/lib/python3.10/site-packages/astropy/time/utils.py:181: RuntimeWarning: invalid value encountered in double_scalars
  ah = c - abig
/home/lingrui/.local/lib/python3.10/site-packages/astropy/time/utils.py:179: RuntimeWarning: overflow encountered in double_scalars
  c = 134217729. * a  # 2**27+1.
/home/lingrui/.local/lib/python3.10/site-packages/astropy/time/utils.py:181: RuntimeWarning: invalid value encountered in double_scalars
  ah = c - abig
/home/lingrui/.local/lib/python3.10/site-packages/astropy/time/utils.py:179: RuntimeWarning: overflow encountered in double_scalars
  c = 134217729. * a  # 2**27+1.
/home/lingrui/.local/lib/python3.10/site-packages/astropy/time/utils.py:181: RuntimeWarning: invalid value encountered in double_scalars
  ah = c - abig
/home/lingrui/.local/lib/python3.10/site-packa

Dealing with Tsys_psr_Sun17_A_10_M01-M01_211119_071000to211119_072348.fits
Dealing with Tsys_psr_Sun17_A_10_M02-M02_211119_071000to211119_072348.fits
Dealing with Tsys_psr_Sun17_A_10_M03-M03_211119_071000to211119_072348.fits
Dealing with Tsys_psr_Sun17_A_10_M04-M04_211119_071000to211119_072348.fits
Dealing with Tsys_psr_Sun17_A_10_M05-M05_211119_071000to211119_072348.fits

Dealing with Tsys_psr_Sun17_A_10_M06-M06_211119_071000to211119_072348.fitsDealing with Tsys_psr_Sun17_A_10_M07-M07_211119_071000to211119_072348.fits
Dealing with Tsys_psr_Sun17_A_10_M08-M08_211119_071000to211119_072348.fits
Dealing with Tsys_psr_Sun17_A_10_M09-M09_211119_071000to211119_072348.fits
Dealing with Tsys_psr_Sun17_A_10_M10-M10_211119_071000to211119_072348.fits
Dealing with Tsys_psr_Sun17_A_10_M11-M11_211119_071000to211119_072348.fits
Dealing with Tsys_psr_Sun17_A_10_M12-M12_211119_071000to211119_072348.fits
Dealing with Tsys_psr_Sun17_A_10_M13-M13_211119_071000to211119_072348.fits
Dealing with Tsys_psr_Sun

/home/lingrui/.local/lib/python3.10/site-packages/astropy/time/utils.py:179: RuntimeWarning: overflow encountered in double_scalars
  c = 134217729. * a  # 2**27+1.
/home/lingrui/.local/lib/python3.10/site-packages/astropy/time/utils.py:181: RuntimeWarning: invalid value encountered in double_scalars
  ah = c - abig
/home/lingrui/.local/lib/python3.10/site-packages/astropy/time/utils.py:179: RuntimeWarning: overflow encountered in double_scalars
  c = 134217729. * a  # 2**27+1.
/home/lingrui/.local/lib/python3.10/site-packages/astropy/time/utils.py:181: RuntimeWarning: invalid value encountered in double_scalars
  ah = c - abig
/home/lingrui/.local/lib/python3.10/site-packages/astropy/time/utils.py:179: RuntimeWarning: overflow encountered in double_scalars
  c = 134217729. * a  # 2**27+1.
/home/lingrui/.local/lib/python3.10/site-packages/astropy/time/utils.py:181: RuntimeWarning: invalid value encountered in double_scalars
  ah = c - abig
/home/lingrui/.local/lib/python3.10/site-packa

#### gridding

In [7]:
os.system('ls $PWD/*/*/Tsys_psr*icrs.fits > mygridcont_list')

0

In [2]:
from gridding_with_doppler_tracking import gridding_with_doppler_tracking

speclist = 'mygridcont_list'
center = [284.5,6.5]
outname = 'Sun17_A_scans_all_doppler_cont.fits'
vmin = (1.4204-1.45)/1.4204*3e5
vmax = (1.4204-1.05)/1.4204*3e5
dv   = 500e-3/1024/1.4204*3e5 
gridding_with_doppler_tracking(speclist=speclist, center=center, imsize=[360, 360], \
                               cell=1/60, rest_freq=1.420405751768, vmin=vmin, vmax=vmax, \
                               dv=dv, p=0,  outname=outname, psrflag=True, overwrite=False)

{'NAXIS': 2, 'NAXIS1': 360, 'NAXIS2': 360, 'CTYPE1': 'RA---SIN', 'CTYPE2': 'DEC--SIN', 'CUNIT1': 'deg', 'CUNIT2': 'deg', 'CDELT1': -0.016666666666666666, 'CDELT2': 0.016666666666666666, 'CRPIX1': 180.0, 'CRPIX2': 180.0, 'CRVAL1': 284.5, 'CRVAL2': 6.5}
Gridding /bb8/lingrui/my_projects/FAST_pipeline/Sun17_A_00/M01/Tsys_psr_Sun17_A_00_M01-M01_211010_100500to211010_101848_icrs.fits
Doing Doppler Tracking!


/home/lingrui/.local/lib/python3.10/site-packages/astropy/time/utils.py:179: RuntimeWarning: overflow encountered in double_scalars
  c = 134217729. * a  # 2**27+1.
/home/lingrui/.local/lib/python3.10/site-packages/astropy/time/utils.py:181: RuntimeWarning: invalid value encountered in double_scalars
  ah = c - abig


Gridding /bb8/lingrui/my_projects/FAST_pipeline/Sun17_A_00/M02/Tsys_psr_Sun17_A_00_M02-M02_211010_100500to211010_101848_icrs.fits
Doing Doppler Tracking!


Gridding /bb8/lingrui/my_projects/FAST_pipeline/Sun17_A_00/M03/Tsys_psr_Sun17_A_00_M03-M03_211010_100500to211010_101848_icrs.fits
Doing Doppler Tracking!


Gridding /bb8/lingrui/my_projects/FAST_pipeline/Sun17_A_00/M04/Tsys_psr_Sun17_A_00_M04-M04_211010_100500to211010_101848_icrs.fits
Doing Doppler Tracking!


...

Gridding /bb8/lingrui/my_projects/FAST_pipeline/Sun17_A_10/M16/Tsys_psr_Sun17_A_10_M16-M16_211119_071000to211119_072348_icrs.fits
Doing Doppler Tracking!


Gridding /bb8/lingrui/my_projects/FAST_pipeline/Sun17_A_10/M17/Tsys_psr_Sun17_A_10_M17-M17_211119_071000to211119_072348_icrs.fits
Doing Doppler Tracking!


Gridding /bb8/lingrui/my_projects/FAST_pipeline/Sun17_A_10/M18/Tsys_psr_Sun17_A_10_M18-M18_211119_071000to211119_072348_icrs.fits
Doing Doppler Tracking!


Gridding /bb8/lingrui/my_projects/FAST_pipeline/Sun17_A_10/M19/Tsys_psr_Sun17_A_10_M19-M19_211119_071000to211119_072348_icrs.fits
Doing Doppler Tracking!


/bb8/lingrui/my_projects/FAST_pipeline/FISH_scripts/gridding_with_doppler_tracking.py:146: RuntimeWarning: invalid value encountered in true_divide
  gridded_data = gridder.get_datacube()
